In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ieee-cis-12nd-solution-part-3-cat/__results__.html
/kaggle/input/ieee-cis-12nd-solution-part-3-cat/custom.css
/kaggle/input/ieee-cis-12nd-solution-part-3-cat/cat_train.csv
/kaggle/input/ieee-cis-12nd-solution-part-3-cat/cat.csv
/kaggle/input/ieee-cis-12nd-solution-part-3-cat/__notebook__.ipynb
/kaggle/input/ieee-cis-12nd-solution-part-3-cat/__output__.json
/kaggle/input/ieee-cis-12nd-solution-part-3-cat/__resultx__.html
/kaggle/input/ieee-cis-12nd-solution-part-3-cat/catboost_info/catboost_training.json
/kaggle/input/ieee-cis-12nd-solution-part-3-cat/catboost_info/test_error.tsv
/kaggle/input/ieee-cis-12nd-solution-part-3-cat/catboost_info/time_left.tsv
/kaggle/input/ieee-cis-12nd-solution-part-3-cat/catboost_info/learn_error.tsv
/kaggle/input/ieee-cis-12nd-solution-part-3-cat/catboost_info/test/events.out.tfevents
/kaggle/input/ieee-cis-12nd-solution-part-3-cat/catboost_info/learn/events.out.tfevents
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [3]:
y_train = train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv',usecols = ['isFraud'])
y_train = y_train.isFraud

In [4]:
lgb = pd.read_csv('/kaggle/input/ieee-cis-12nd-solution-part-2-lgb/lgb.csv')
lgb_train = pd.read_csv('/kaggle/input/ieee-cis-12nd-solution-part-2-lgb/lgb_train.csv')
xgb = pd.read_csv('/kaggle/input/ieee-cis-12nd-solution-part-4-xgb/xgb.csv')
xgb_train = pd.read_csv('/kaggle/input/ieee-cis-12nd-solution-part-4-xgb/xgb_train.csv')
cat = pd.read_csv('/kaggle/input/ieee-cis-12nd-solution-part-3-cat/cat.csv')
cat_train = pd.read_csv('/kaggle/input/ieee-cis-12nd-solution-part-3-cat/cat_train.csv')

In [5]:
# 先构造一个矩阵
df = np.array([lgb.isFraud, xgb.isFraud, cat.isFraud])
# 计算协方差矩阵
np.corrcoef(df)

array([[1.        , 0.99528438, 0.92901527],
       [0.99528438, 1.        , 0.93148535],
       [0.92901527, 0.93148535, 1.        ]])

In [6]:
print(roc_auc_score(y_train.values,lgb_train.train.values))
print(roc_auc_score(y_train.values,xgb_train.train.values))
print(roc_auc_score(y_train.values,cat_train.train.values))

0.9550520141936576
0.9525709411706289
0.9595312371767415


由于kernal notebook的机能限制我们这里的catboost仅使用了0.1的学习率，如果使用0.05的学习率，会有更高的成绩

                cv       public  private
            lbg 95505    96095   93860
            XGB 95257    96000   93509
            cat 95953    95858   93302  
    0.9*lgb + 0.1*xgb =  96099   93839
    0.9*lgb + 0.1*cat =  96292   94113      
    0.8*lgb + 0.2*cat =  96321   94119        <-- this is our bst score . 
                                         if lr of catboost is 0.005,private score could be 9415
    0.7*lgb + 0.3*cat =  96314   94082

In [7]:
sub = lgb.copy()
sub.isFraud = 0.8*lgb.isFraud + 0.2*cat.isFraud
sub.to_csv('./sub.csv',index = None)